# A.10 NumPy的结构化数据

### 10.1 引言

- 问题 —— NumPy 处理结构化数据（混合、异质）

    - 结构化数组
    - 记录数组


- 参考 —— Pandas 数据

    - Dataframe
    - TimeSeries

In [1]:
import numpy as np

### 10.2  实例 —— 关于人的多种类数据

- 姓名
- 年龄
- 体重

可采用三个不同的数组

In [2]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

#### 评价

- 上述方式 —— 笨拙，三个数组缺少关联
- 改进方式 —— 用一个结构存贮所有数据

#### NumPy 提供的结构化类型

- 专门用于复合型数据

#### 注意下例中的 ``dtype``

In [3]:
x = np.zeros(4, dtype=int)
x

array([0, 0, 0, 0])

#### 使用复合数据类型

- 创建结构化数组
- 注意 zeros 函数的作用

In [4]:
data = np.zeros(4, dtype={'names':('name', 'age', 'weight'),
                          'formats':('U10', 'i4', 'f8')})
print("data.dtype\n", data.dtype)
print("data\n",data)

data.dtype
 [('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]
data
 [('', 0, 0.) ('', 0, 0.) ('', 0, 0.) ('', 0, 0.)]


#### 说明 —— 数据结构

- ``'U10'`` 指“最大长度10的 Unicode 字符串”
- ``'i4'`` 指“4字节（即32位）整数”
- ``'f8'`` 指“8字节（即64位）浮点数”

现在创建了一空容器数组，可以用列表来填充数组

In [5]:
data['name'] = name
data['age'] = age
data['weight'] = weight

print("data\n", data)

data
 [('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]


#### 复合之后

- 数据现在整合在了一起
- 结构化数组使得可以通过索引或名称来引用元素值

In [6]:
# 获得所有名称
data['name']

array(['Alice', 'Bob', 'Cathy', 'Doug'], dtype='<U10')

In [7]:
# 获得首条数据(记录)
data[0]

('Alice', 25, 55.)

In [8]:
# 获得末行名称
data[-1]['name']

'Doug'

#### 布尔掩码 —— 更复杂的操作

- 如过滤等

In [15]:
# 获得年龄小于 30 的人名
data[data['age'] < 30]['name']

array(['Alice', 'Doug'], dtype='<U10')

### 10.3 与 Pandas 相结合

- 考虑使用 Pandas 包
- Pandas 的 ``Dataframe`` 对象

#### 创建结构化数组

结构化数组的数据类型可有许多方式来指定

- 字典方法(早期用)

In [10]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

#### 使用``dtype``关键字

为了表述清楚，数值类型可采用

- Python 类型

或

- NumPy 的``dtype``类型

来指明

In [11]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':((np.str_, 10), int, np.float32)})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f4')])

- 元组列表 —— 指定复合类型

In [12]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

dtype([('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

- 逗号分隔的字符串 —— 指定复合类型

In [13]:
np.dtype('S10,i4,f8')

dtype([('f0', 'S10'), ('f1', '<i4'), ('f2', '<f8')])

### 10.4 数据类型定义的原则

缩短了的字符格式代码容易引起困惑，不过总有一些原则遵循

- 第一个（可选）字符是``<``或``>``，分别表示"little endian"或"big endian"
- 下一个字符指定数据类型：字符、字节、整型、浮点型等（见下表）
- 最后一个（或多个）字符表示对象占用的字节数

| 字符        | 描述           | 示例                             |
| ---------        | -----------           | -------                             | 
| ``'b'``          | Byte                  | ``np.dtype('b')``                   |
| ``'i'``          | Signed integer        | ``np.dtype('i4') == np.int32``      |
| ``'u'``          | Unsigned integer      | ``np.dtype('u1') == np.uint8``      |
| ``'f'``          | Floating point        | ``np.dtype('f8') == np.int64``      |
| ``'c'``          | Complex floating point| ``np.dtype('c16') == np.complex128``|
| ``'S'``, ``'a'`` | String                | ``np.dtype('S5')``                  |
| ``'U'``          | Unicode string        | ``np.dtype('U') == np.str_``        |
| ``'V'``          | Raw data (void)       | ``np.dtype('V') == np.void``        |

### 10.5 更高级的复合类型

- 有可能定义更高级的复合类型，如某复合类型的每个元素是一个数组或矩阵

如下例，创建一个带``mat``组件由$3\times 3$浮点矩阵构成的数据类型

In [16]:
tp = np.dtype([('id', 'i8'), ('mat', 'f8', (3, 3))])
X = np.zeros(1, dtype=tp)
print(X[0])
print(X['mat'][0])

(0, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
[[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


#### ``X``数组每一元素的组成成份

- ``id``，8位整型
- ``mat``，$3\times 3$ 矩阵

#### 为什么这样定义？

- NumPy 的 ``dtype`` 直接映射成 C 语言结构定义
- 便于通过恰当编写的 C 程序直接访问

### 结束